In [2]:
import pandas as pd
from pathlib import Path
import sqlite3

pd.set_option('display.max_columns', None)

#sqlite in RAM memory decleration 
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
#Python lib to resolve directory
scriptDir = Path.cwd()
dataDir = scriptDir.parent / "data"

In [3]:
customers = pd.read_csv(dataDir/"customers.csv")
articles = pd.read_csv(dataDir/"articles.csv")
transactions = pd.read_csv(dataDir/"transactions_train.csv")  # Update path if needed

transactions.head().style.set_caption("Transactions")

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,685687004,0.016932,2


In [7]:
customers.head().style.set_caption("Customers")


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,nan,nan,ACTIVE,NONE,49.000000,52043ee2162cf5aa7ee79974281641c6f11a68d276429a91f8ca0d4b6efa8100
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,nan,nan,ACTIVE,NONE,25.000000,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93f4c830291c32bc3057
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,nan,nan,ACTIVE,NONE,24.000000,64f17e6a330a85798e4998f62d0930d14db8db1c054af6c9090f7dd3e38380dc
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,nan,nan,ACTIVE,NONE,54.000000,5d36574f52495e81f019b680c843c443bd343d5ca5b1c222539af5973a23ae6d
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,1.000000,1.000000,ACTIVE,Regularly,52.000000,25fa5ddee9aac01b35208d01736e57942317d756b32ddd4564743b005a805b1d


In [10]:
articles.head().style.set_caption("Articles")


,Unnamed: 0,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,1,Dusty Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,4,Dark,5,Black,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulded, lightly padded cups that shape the bust and provide good support. Narrow adjustable shoulder straps and a narrow hook-and-eye fastening at the back. Without visible seams for greater comfort."
3,4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,3,Light,9,White,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulded, lightly padded cups that shape the bust and provide good support. Narrow adjustable shoulder straps and a narrow hook-and-eye fastening at the back. Without visible seams for greater comfort."
4,8,111586001,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,4,Dark,5,Black,3608,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.


In [12]:
customers.to_sql('customers',conn,False,'replace')
articles.to_sql('articles',conn,False,'replace')
transactions.to_sql('transactions',conn,False,'replace')

sqlCustomerTransactionsQuery = (
    "CREATE TABLE tblCustomerTransactions AS "
    "SELECT c.customer_id, "
    "c.fn, "
    "c.active, "
    "c.club_member_status, "
    "c.fashion_news_frequency, "
    "c.age, "
    "c.postal_code, "
    "t.t_dat AS trans_date, "
    "t.article_id AS trans_article_id, "
    "t.price AS price "
    "FROM customers AS c, articles AS a, transactions AS t "
    "WHERE t.customer_id = c.customer_id and t.t_dat = '2018-09-20' "
)
#Create tblCustomerTransactions of joined Customers and Transactions
cursor.execute("DROP TABLE IF EXISTS tblCustomerTransactions;")
cursor.execute(sqlCustomerTransactionsQuery)

sqlCustomersTransactionsTable = pd.read_sql_query('SELECT * FROM tblCustomerTransactions LIMIT 2000;',conn)



In [13]:
sqlCustomersTransactionsTable.head()
len(sqlCustomersTransactionsTable)

2000

In [15]:
sqlCompleteDataQuery = (
    "CREATE TABLE tblCompleteData AS "
    "SELECT cs.customer_id, "
    "cs.trans_date, "
    "a.article_id, "
    "a.product_type_name, "
    "cs.price, "
    "cs.age, "
    "cs.fashion_news_frequency "
    "FROM articles AS a, tblCustomerTransactions AS cs "
    "WHERE cs.trans_article_id = a.article_id;"
)
cursor.execute("DROP TABLE IF EXISTS tblCompleteData;")
cursor.execute(sqlCompleteDataQuery)

dfTBLCompleteData = pd.read_sql_query('SELECT * FROM tblCompleteData ;',conn)


In [16]:
dfTBLCompleteData.tail().style.set_caption("Final Test Output")

,customer_id,trans_date,article_id,product_type_name,price,age,fashion_news_frequency
1637597,01c290bff9641834691fef7f4b68ad176353d6971ca85e22efeb92c3aa3ffd35,2018-09-20,714417001,Blouse,0.025407,21.000000,Regularly
1637598,01c290bff9641834691fef7f4b68ad176353d6971ca85e22efeb92c3aa3ffd35,2018-09-20,714417001,Blouse,0.025407,21.000000,Regularly
1637599,01c290bff9641834691fef7f4b68ad176353d6971ca85e22efeb92c3aa3ffd35,2018-09-20,714417001,Blouse,0.025407,21.000000,Regularly
1637600,01c290bff9641834691fef7f4b68ad176353d6971ca85e22efeb92c3aa3ffd35,2018-09-20,714417001,Blouse,0.025407,21.000000,Regularly
1637601,01c290bff9641834691fef7f4b68ad176353d6971ca85e22efeb92c3aa3ffd35,2018-09-20,714417001,Blouse,0.025407,21.000000,Regularly


In [32]:
checkingStr = dfTBLCompleteData['article_id'].value_counts()
print(checkingStr)

685687004    19612
631848002    19612
662888002    14709
651456003    14709
692454002    14709
             ...  
590920002     4903
590951007     4903
591334019     4903
591702003     4903
714417001     4903
Name: article_id, Length: 283, dtype: int64


In [35]:
len(dfTBLCompleteData)

1637602